In [22]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import metrics
from tensorflow.keras.optimizers import Adam
from sklearn.svm import LinearSVC
from keras.models import Model
from sklearn import svm
import itertools
from sklearn.cluster import KMeans
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from keras.models import load_model

In [23]:
# Image data preprocecing
train_path = './Train'
test_path = './Test'
imgDataGen = ImageDataGenerator(rescale=1.0/255)
train = imgDataGen.flow_from_directory(train_path,  color_mode ='grayscale', target_size = (64, 64), batch_size = 1, class_mode = 'binary')
print(train[0][0].shape)
test = imgDataGen.flow_from_directory(test_path,  color_mode ='grayscale', target_size = (64, 64), batch_size = 1, class_mode = 'binary')

Found 24959 images belonging to 2 classes.
(1, 64, 64, 1)
Found 9471 images belonging to 2 classes.


In [24]:
train_y = []

for i in range(len(train)):
    train_y.append(train[i][1])
train_y = list(itertools.chain(*train_y))
print(len(train_y))

test_y = []
for i in range(len(test)):
    test_y.append(test[i][1])
test_y = list(itertools.chain(*test_y))
print(len(test_y))

/Users/zyt/opt/anaconda3/envs/my-env/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:845: UserWarning: Truncated File Read
  warnings.warn(str(msg))


24959
9471


In [25]:
myModel = tf.keras.models.load_model('DogVsCat')

In [26]:
model2= Model(inputs=myModel.input, outputs=myModel.layers[-6].output)

In [27]:

predictions_train = model2.predict(train)
predictions_test = model2.predict(test)

22843/24959 [==========================>...] - ETA: 5s

/Users/zyt/opt/anaconda3/envs/my-env/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:845: UserWarning: Truncated File Read
  warnings.warn(str(msg))


9471/9471 [==============================] - 23s 2ms/step


In [28]:
print(predictions_train.shape)
print(predictions_test.shape)

(24959, 128)
(9471, 128)


In [29]:
classifer = LinearSVC(C = 1e-9, max_iter=10000)
classifer.fit(predictions_train,train_y)
pred = classifer.predict(predictions_test)
print("Accuracy LinearSVC(%):",metrics.accuracy_score(pred, test_y)*100,"%")

Accuracy LinearSVC(%): 84.4789356984479 %


In [30]:
classifer = svm.SVC(C = 1, max_iter=10000, kernel ='rbf', gamma = 0.04)
classifer.fit(predictions_train,train_y)
score = classifer.score(predictions_test,test_y)
print("Accuracy SVM(%):",score*100,"%")

Accuracy SVM(%): 95.79769823672262 %


In [34]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(predictions_train)
pred = kmeans.predict(predictions_test)
print("Accuracy (%):",metrics.accuracy_score(pred, test_y)*100,"%")


Accuracy (%): 80.51948051948052 %


In [32]:
xgbclf=xgb.XGBClassifier()
xgbclf.fit(predictions_train,train_y)
predict=xgbclf.predict(predictions_test)
mixed=xgbclf.score(predictions_test,test_y)
print("Accuracy (%) XGB: ", mixed)

Accuracy (%) XGB:  0.977299123640587


In [33]:
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(predictions_train, train_y)
result=clf.score(predictions_test, test_y)
print("Accuracy ADA(%):",result*100,"%")

Accuracy ADA(%): 93.03135888501743 %
